In [24]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import datetime as dt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
import random
import pprint
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

#### Reading data from preprocessed file.

In [25]:
data = pd.read_csv('../data/prepocessed.csv', encoding='utf8', engine='python')
data = data.drop(data.columns[0], axis=1)

print(data.head())

   State Code  County Code  Site Num                                  Address  \
0           4           13      3002  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN   
1           4           13      3002  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN   
2           4           13      3002  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN   
3           4           13      3002  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN   
4           4           13      3002  1645 E ROOSEVELT ST-CENTRAL PHOENIX STN   

     State    County     City  Date Local          NO2 Units   NO2 Mean  ...  \
0  Arizona  Maricopa  Phoenix  2000-01-01  Parts per billion  19.041667  ...   
1  Arizona  Maricopa  Phoenix  2000-01-01  Parts per billion  19.041667  ...   
2  Arizona  Maricopa  Phoenix  2000-01-01  Parts per billion  19.041667  ...   
3  Arizona  Maricopa  Phoenix  2000-01-01  Parts per billion  19.041667  ...   
4  Arizona  Maricopa  Phoenix  2000-01-02  Parts per billion  22.958333  ...   

           SO2 Units  SO2 Mean  

#### Creating separate files of timeseries for each state.

In [26]:
for state in states:
    filename = "state Data/"+state + ".csv"
    datatemp = data.loc[data['State'] == state]
    datatemp = datatemp.groupby(['Date Local']).mean().reset_index()
    datatemp = datatemp[['Date Local','NO2 Mean']].copy()
    datatemp.to_csv(filename, ',', encoding='utf8')

FileNotFoundError: [Errno 2] No such file or directory: 'state Data/Arizona.csv'

#### creating list of timeseries', later to be fed in algorithm

In [ ]:
timeSeriesList = []
j=0
for state in states:
    filename = "../state Data/"+state+".csv"
    datatemp = pd.read_csv(filename, usecols=['NO2 Mean'])
    dataList = datatemp['NO2 Mean'].tolist()
    seriesNP = np.asarray(dataList)
    timeSeriesList.append(dataList[0:200])
    j=j+1
finaltimeSeriesList = np.asarray(timeSeriesList)

#### Dynamic Time Wrapping & LB_Keogh for lower bounding of DTW

In [ ]:
def DynamicTimeWarping(timeseries, centroid_timeseries, window):
    DTW_Dictionary={}
    
    for i in range(0,(len(timeseries)+1)):
        for j in range(0,(len(centroid_timeseries)+1)):
            if((i==0) & (j==0)):
                DTW_Dictionary[(i, j)] = float(0)
            else:
                DTW_Dictionary[(i, j)] = float('inf')    
       
    for item in range(len(timeseries)):
        low = (item-window) if (item-window)>=0 else 0
        high = (len(centroid_timeseries)) if ((item+window) >= (len(centroid_timeseries))) else (item+window)
        for second_item in range(low, high):    
            distance= (timeseries[item]-centroid_timeseries[second_item])**2
            DTW_Dictionary[(item+1), (second_item+1)] = distance + min(DTW_Dictionary[(item, second_item+1)],DTW_Dictionary[(item+1, second_item)], DTW_Dictionary[(item, second_item)])
    return np.sqrt(DTW_Dictionary[len(timeseries), len(centroid_timeseries)])

def LowerBoundKeogh(timeseries,centroid_timeseries,reach):
    lowerBoundSum=0
    for index, item in enumerate(timeseries):
        low = 0 if (index-reach)<0 else (index-reach)
        if len(centroid_timeseries[(low):(index+reach)])==0:
            break
        lowerBound = min(centroid_timeseries[(low):(index+reach)])
        upperBound = max(centroid_timeseries[(low):(index+reach)])
        
        if item>upperBound:
            lowerBoundSum = lowerBoundSum+(item-upperBound)**2
        elif item<lowerBound:
            lowerBoundSum = lowerBoundSum+(item-lowerBound)**2
    
    return np.sqrt(lowerBoundSum)

#### K-Means Clustering. Args - Data, number of clusters, number of iterations, window for DynamicTimeWarping

In [ ]:
def KMeans(data, numCluster, iterations, window):
    centroids = random.sample(list(data),numCluster)

    for n in range(iterations):
        print(n)
        clusterID = {}
        
        for index, item in enumerate(data):
            minDist = float('inf')
            closestCluster = None
            for centroidIndex, j in enumerate(centroids):
                if LowerBoundKeogh(item, j, 5) < minDist:
                    currentDist = DynamicTimeWarping(item, j, window)
                    if currentDist < minDist:
                        minDist = currentDist
                        closestCluster = centroidIndex
                       
            if closestCluster in clusterID:
                clusterID[closestCluster].append(index)
            else:
                clusterID[closestCluster]=[]

        for ID in clusterID:
            clusterSum = [0]*200
            if ID is not None:
                for k in clusterID[ID]:
                    clusterSum = clusterSum + data[k]   
                centroids[ID] = [m/len(clusterID[ID]) for m in clusterSum if len(clusterID[ID])!=0]
    
    return centroids, clusterID

In [ ]:
centroids, clusterID = KMeans(finaltimeSeriesList, 4, 10, 5)

for i in centroids:
    plt.plot(i)
    
plt.show()

#### Results of clustering state-wise

In [ ]:
dictionary = {}
for i in clusterID:
    list = clusterID[i]
    tempList = []
    for state in list:
        tempList.append(states[state])
    dictionary[i] = tempList
pprint.pprint(dictionary)

#### Converting cluster centroids into time-series

In [ ]:
base = dt.datetime(2000, 1, 1, 0, 0, 0)
datelist = pd.date_range(base, periods=200).tolist()

cluster1 = pd.DataFrame({'Date':datelist, 'NO2 Mean':centroids[0]})
cluster1['Date'] = pd.to_datetime(cluster1['Date'], format='%Y-%m-%d')


cluster2 = pd.DataFrame({'Date':datelist, 'NO2 Mean':centroids[1]})
cluster2['Date'] = pd.to_datetime(cluster2['Date'], format='%Y-%m-%d')

cluster3 = pd.DataFrame({'Date':datelist, 'NO2 Mean':centroids[2]})
cluster3['Date'] = pd.to_datetime(cluster3['Date'], format='%Y-%m-%d')

cluster4 = pd.DataFrame({'Date':datelist, 'NO2 Mean':centroids[3]})
cluster4['Date'] = pd.to_datetime(cluster4['Date'], format='%Y-%m-%d')

cluster1TS = cluster1.set_index('Date')
cluster2TS = cluster2.set_index('Date')
cluster3TS = cluster3.set_index('Date')
cluster4TS = cluster4.set_index('Date')

#### Generating graph for each clusters

In [ ]:
label=", ".join(str(x) for x in dictionary[0])
plt.plot(cluster1TS, label=label)
plt.legend(loc="best")
plt.title("Cluster 1")
plt.show(block=False)

In [ ]:
label=", ".join(str(x) for x in dictionary[1])
plt.plot(cluster2TS, label=label, color='red')
plt.legend(loc="best")
plt.title("Cluster 2")
plt.show(block=False)

In [ ]:
label=", ".join(str(x) for x in dictionary[2])
plt.plot(cluster3TS, label=label, color='black')
plt.legend(loc="best")
plt.title("Cluster 3")
plt.show(block=False)

In [ ]:
label=", ".join(str(x) for x in dictionary[3])
plt.plot(cluster4TS, label=label, color='green')
plt.legend(loc="best")
plt.title("Cluster 4")
plt.show(block=False)

#### ARIMA on cluster 1

In [ ]:
cluster1Values = cluster1TS.values
train, test = cluster1Values[0:int(len(cluster1Values) * 0.80)], cluster1Values[int(len(cluster1Values) * 0.80):len(cluster1Values)]
trainingData = [x for x in train]
predictions = []

for item in range(len(test)):
    model = ARIMA(trainingData, order=(5,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    predicted = output[0]
    predictions.append(predicted)
    observation = test[item]
    trainingData.append(observation)
    print('predicted=%f, expected=%f' % (predicted, observation))

In [ ]:
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)

plt.plot(test)
plt.plot(predictions, color='red')
plt.show()

In [ ]:
intraClusterError = {}
interDistance = 0

for j in clusterID.keys():
    distance = 0
    for i in clusterID[j]:
        temp = finaltimeSeriesList[j][0:200]
        tempCentroid = centroids[j]
        distance = distance + DynamicTimeWarping(temp, tempCentroid, 5)
    intraClusterError[j] = distance
    
intraDistance = sum(intraClusterError.values())

for i in range(4):
    for j in range(i+1,4):
        interDistance = interDistance + DynamicTimeWarping(centroids[i], centroids[j], 5)
        
variance = interDistance + intraDistance

print('Intra Cluster Distance: %.3f' % intraDistance)
print('Inter Cluster Distance: %.3f' % interDistance)
print('Total Sum Square (Variance): %.3f' % variance)
